In [122]:
# Loading all the libraries

import boto3
import json
import cv2
import decimal
from copy import deepcopy

from __future__ import print_function
import base64
import datetime
import time
import decimal
import uuid
import json
import boto3
import pytz
from pytz import timezone
from copy import deepcopy

from PIL import Image, ImageDraw, ExifTags, ImageColor, ImageFont



In [123]:
# Initializing the AWS Session


AWS_ACCESS_KEY_ID = 'XXXXXXXXXXXXXXXXXXX'
AWS_SECRET_ACCESS_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXX'


session = boto3.session.Session(aws_access_key_id = AWS_ACCESS_KEY_ID,
                                aws_secret_access_key = AWS_SECRET_ACCESS_KEY,
                                region_name='us-west-2')

In [124]:
# Reading the Image Frame

frame = cv2.imread('person.jpg', 0)
retval, buff = cv2.imencode(".jpg", frame)
img_bytes = bytearray(buff)

image=Image.open('person.jpg')

imgWidth, imgHeight = image.size  
draw = ImageDraw.Draw(image)  



In [125]:
def load_config():
    '''Load configuration from file.'''
    with open('image-processor.json', 'r') as conf_file:
        conf_json = conf_file.read()
        return json.loads(conf_json)

#Load config
config = load_config()


rekog_client = session.client('rekognition')


rekog_max_labels = config["rekog_max_labels"]
rekog_min_conf = float(config["rekog_min_conf"])

label_watch_list = config["label_watch_list"]
label_watch_min_conf = float(config["label_watch_min_conf"])
label_watch_phone_num = config.get("label_watch_phone_num", "")
label_watch_sns_topic_arn = config.get("label_watch_sns_topic_arn", "")

In [126]:
rekog_response = rekog_client.detect_labels(
            Image={
                'Bytes': img_bytes
            },
            MaxLabels=rekog_max_labels,
            MinConfidence=rekog_min_conf
        )

In [127]:
boxes = []

objects = []

for label in rekog_response['Labels']:
        print ("Label: " + label['Name'])
        print ("Confidence: " + str(label['Confidence']))
        print ("Instances:")
        for instance in label['Instances']:
            print ("  Bounding box")
            
            if(instance['BoundingBox']['Top'] > 0):
                top = imgHeight * instance['BoundingBox']['Top']
                left = imgWidth * instance['BoundingBox']['Left']
                width = imgWidth * instance['BoundingBox']['Width']
                height = imgHeight * instance['BoundingBox']['Height']
            
                boxes.append([top,left,width,height])
                
                objects.append(label['Name'])
            
            print ("    Top: " + str(instance['BoundingBox']['Top']))
            print ("    Left: " + str(instance['BoundingBox']['Left']))
            print ("    Width: " +  str(instance['BoundingBox']['Width']))
            print ("    Height: " +  str(instance['BoundingBox']['Height']))
            print ("  Confidence: " + str(instance['Confidence']))
            print()

        print ("Parents:")
        for parent in label['Parents']:
            print ("   " + parent['Name'])
        print ("----------")

Label: Road
Confidence: 99.9649658203125
Instances:
Parents:
----------
Label: Intersection
Confidence: 99.9649658203125
Instances:
Parents:
   Road
----------
Label: Outdoors
Confidence: 97.76097106933594
Instances:
Parents:
----------
Label: Landscape
Confidence: 97.76097106933594
Instances:
Parents:
   Nature
   Outdoors
----------
Label: Nature
Confidence: 97.76097106933594
Instances:
Parents:
----------
Label: Asphalt
Confidence: 96.86266326904297
Instances:
Parents:
----------
Label: Tarmac
Confidence: 96.86266326904297
Instances:
Parents:
----------
Label: Automobile
Confidence: 96.75332641601562
Instances:
Parents:
   Transportation
   Vehicle
----------
Label: Car
Confidence: 96.75332641601562
Instances:
  Bounding box
    Top: 0.47980308532714844
    Left: 0.7881690263748169
    Width: 0.0785754844546318
    Height: 0.0992165133357048
  Confidence: 96.75332641601562

  Bounding box
    Top: 0.09746074676513672
    Left: 0.2758864462375641
    Width: 0.07413380593061447
    He

In [128]:
boxes

[[435.18139839172363, 720.3864901065826, 71.81799279153347, 89.98937759548426],
 [88.396897315979, 252.16021186113358, 67.75829862058163, 38.820219960063696],
 [60.47077172994614, 161.3284704387188, 83.84057858586311, 54.32524264231324],
 [351.13568794727325, 756.192688703537, 57.7653828561306, 76.89648730307817],
 [90.93257354944944, 761.653682589531, 47.01940877735615, 36.74785003438592],
 [364.9563259482384, 674.6767820119858, 69.18017764389515, 87.38757073879242],
 [295.2913838028908, 634.9839134216309, 69.45983695983887, 82.36755634099245],
 [763.6886007785797, 13.838493537157774, 168.1616452038288, 97.62024062126875],
 [379.1690713763237, 589.9548070430756, 80.88476477563381, 104.60107597708702],
 [144.05222979187965,
  726.4566642045975,
  52.783569514751434,
  54.14855980873108],
 [262.49332478642464, 720.9042006731033, 65.95593392848969, 72.20068595558405],
 [362.17059540748596,
  838.7648818492889,
  33.13171251118183,
  18.956482354551554],
 [196.3400228023529, 672.344605684

In [129]:
# image=Image.open('pet.jpg')

# imgWidth, imgHeight = image.size  
# draw = ImageDraw.Draw(image) 


for i, box in enumerate(boxes):
    
    top = box[0]
    left = box[1]
    width = box[2]
    height = box[3]
    
    
    points = (
                (left,top),
                (left + width, top),
                (left + width, top + height),
                (left , top + height),
                (left, top)

            )
    
    font = ImageFont.truetype("arial.ttf", 25)
    draw.line(points, fill='#00d400', width=3)
    
    label = objects[i]
    color = 'rgb(255,255,0)' # white color
    draw.text((left, top - 25), label, fill=color,font=font)
    

In [130]:
image.show()



In [132]:
import io
frame = cv2.imread('person.jpg', 1)
retval, buff = cv2.imencode(".jpg", frame)
img_bytes = bytearray(buff)
stream = io.BytesIO(img_bytes)
image=Image.open(stream)

image.show()

In [135]:
image.tobytes()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
